# **PYSPARK - MACHINE LEARNING - LINEAR REGRESSION**

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('linregx').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [6]:
# DataFrame
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
# Instanciando modelo
lr = LinearRegression(featuresCol='features', labelCol='label',
                      predictionCol='prediction')

In [8]:
# Fit / Treino
lrModel = lr.fit(training)

In [9]:
# Informações do modelo
print(lrModel.coefficients)
print("\n")
print(lrModel.intercept)

[0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]


0.14228558260358093


In [10]:
# Mais informações
training_summary = lrModel.summary

In [11]:
# R quadrado
training_summary.r2


0.027839179518600154

In [12]:
# Raiz quadrada do erro-médio
training_summary.rootMeanSquaredError

10.16309157133015

# **DIVIDINDO EM TREINO E TESTE**

In [13]:
# DataFrame
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [14]:
# Criando variáveis para receber treino e teste
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [15]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                361|
|   mean|0.09128332358952235|
| stddev|  9.819739287200976|
|    min|-28.046018037776633|
|    max|  23.52945433069272|
+-------+-------------------+



In [16]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                140|
|   mean| 0.6839144293133692|
| stddev| 11.530414937939513|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [17]:
# SET DE TREINO
correct_model = lr.fit(train_data)

In [18]:
# SET DE TESTE
test_results = correct_model.evaluate(test_data)

In [19]:
# RESIDUOS:
# Entre o que o modelo esperava e os rótulos que realmente conheciam os dados
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-26.782411307688847|
| -21.18686760618341|
|-26.317945862382633|
|-20.215923749578998|
| -22.48989888135053|
|-19.931089383270944|
|  -20.5272369438829|
| -20.56813687229521|
|-18.355539024494927|
|-15.389791944300383|
|-17.269485226439905|
| -15.92946007109294|
|  -16.9666867410122|
|-14.139576237293682|
|   -9.8960838162552|
|-13.179342616791049|
|-12.013772533006987|
|-11.287622076654413|
|-11.454345893108407|
|-7.3996052991462395|
+-------------------+
only showing top 20 rows



In [20]:
test_results.rootMeanSquaredError

11.478916875051265

In [21]:
unlabeled_data = test_data.select('features')

In [22]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [23]:
predictions = correct_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.7890675620545782|
|(10,[0,1,2,3,4,5,...| -2.3005725147531044|
|(10,[0,1,2,3,4,5,...|    3.36811992618656|
|(10,[0,1,2,3,4,5,...|  -2.621536667340343|
|(10,[0,1,2,3,4,5,...|  2.2778216223918593|
|(10,[0,1,2,3,4,5,...| -0.1263932325182687|
|(10,[0,1,2,3,4,5,...|   0.642676169609477|
|(10,[0,1,2,3,4,5,...|   1.165800842080658|
|(10,[0,1,2,3,4,5,...| 0.08032545849029657|
|(10,[0,1,2,3,4,5,...| -2.1044084125829614|
|(10,[0,1,2,3,4,5,...|-0.05723550623604...|
|(10,[0,1,2,3,4,5,...| -0.7896367625121483|
|(10,[0,1,2,3,4,5,...|  0.8153373897350894|
|(10,[0,1,2,3,4,5,...| -1.2978085561375359|
|(10,[0,1,2,3,4,5,...| -2.8771431829959977|
|(10,[0,1,2,3,4,5,...|  0.6785688314359944|
|(10,[0,1,2,3,4,5,...|-0.46550767844451024|
|(10,[0,1,2,3,4,5,...| -1.1800343043784474|
|(10,[0,1,2,3,4,5,...| -0.4506410095667066|
|(10,[0,1,2,3,4,5,...| -4.240944